<a href="https://colab.research.google.com/github/FREDY129053/Final_theory_work/blob/main/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Обучение модель для предсказания акций IBM, Yandex, MIME

## Импорт библиотек

In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM

## Работа с датасетом, полученным из БД

In [95]:
# df = pd.read_csv('/content/ibm_data.csv')
# df = pd.read_csv('/content/yandex_data.csv')
df = pd.read_csv('/content/mime_data.csv')

In [96]:
df['Date_point'] = pd.to_datetime(df['Date_point'])

In [97]:
df.head(5)

,Date_point,Name,Open,High,Low,Close,Volume
0,2018-12-31,Mimecast Ltd,33.70,34.00,33.2300,33.63,192189.0
1,2018-12-28,Mimecast Ltd,33.15,33.80,31.7300,33.32,266871.0
2,2018-12-27,Mimecast Ltd,31.92,32.78,31.4900,32.70,266509.0
3,2018-12-26,Mimecast Ltd,31.27,32.55,31.2700,32.51,404624.0
4,2018-12-24,Mimecast Ltd,29.68,31.07,29.6647,30.87,327169.0


In [98]:
df.tail(5)

,Date_point,Name,Open,High,Low,Close,Volume
778,2015-11-25,Mimecast Ltd,10.06,10.35,10.06,10.18,175959.0
779,2015-11-24,Mimecast Ltd,10.05,10.40,10.00,10.21,117354.0
780,2015-11-23,Mimecast Ltd,10.45,10.45,10.00,10.22,296011.0
781,2015-11-20,Mimecast Ltd,10.50,10.65,10.00,10.15,630591.0
782,2015-11-19,Mimecast Ltd,11.00,11.99,9.72,10.10,7131458.0


In [99]:
copy_df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
copy_df.shape

(783, 5)

In [100]:
np_ds_scaled = np.reshape(copy_df, (copy_df.shape[0], copy_df.shape[1]))
np_ds_scaled = np_ds_scaled[::-1]
np_ds_scaled

array([[1.100000e+01, 1.199000e+01, 9.720000e+00, 1.010000e+01,
        7.131458e+06],
       [1.050000e+01, 1.065000e+01, 1.000000e+01, 1.015000e+01,
        6.305910e+05],
       [1.045000e+01, 1.045000e+01, 1.000000e+01, 1.022000e+01,
        2.960110e+05],
       ...,
       [3.192000e+01, 3.278000e+01, 3.149000e+01, 3.270000e+01,
        2.665090e+05],
       [3.315000e+01, 3.380000e+01, 3.173000e+01, 3.332000e+01,
        2.668710e+05],
       [3.370000e+01, 3.400000e+01, 3.323000e+01, 3.363000e+01,
        1.921890e+05]])

## Обучение модели

### Подготовка к обучению

#### Функция формирования датасета.

##### Основа функции была взята с сайта https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/#:~:text=learn%20this%20mapping.-,Vector%20Output%20Model,-Like%20other%20types

Там предсказание для одномерного массива, но функция была чуть переделана.
Результат функции из гайда:
- Входные данные: [10, 20, 30, 40, 50, 60, 70, 80, 90]
- Параметры: n_steps_in, n_steps_out = 3, 2
- X = [[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60], [50, 60, 70]]
- y = [[40, 50], [50, 60], [60, 70], [70, 80], [80, 90]]

Функция работает так же, но для пятимерного массива, т.к. 5 признаков

In [101]:
def split_sequence(sequence, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in range(len(sequence)):
      # find the end of this pattern
      end_ix = i + n_steps_in
      out_end_ix = end_ix + n_steps_out
      # check if we are beyond the sequence
      if out_end_ix > len(sequence):
        break
      # gather input and output parts of the pattern
      seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
      X.append(seq_x)
      tmp = [i[-2] for i in seq_y]
      y.append(tmp)
  return np.array(X), np.array(y)

#### Разбиение данных и нормализация(чуть сложно, тк массив в итоге 3х мерный, а не 2х мерный, как требует MimMaxScaler)

In [102]:
n_steps_in, n_steps_out = 40, 90
x, y = split_sequence(np_ds_scaled, n_steps_in, n_steps_out)

In [103]:
x = x.reshape((x.shape[0], x.shape[1], 5))

In [104]:
splitlimit = int(len(x)*0.8)
x_train, x_test = x[:splitlimit], x[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]

In [105]:
from sklearn.preprocessing import MinMaxScaler

normalizer1 = MinMaxScaler(feature_range=(0,1))
normalizer2 = MinMaxScaler(feature_range=(0,1))

x_train_tmp = x_train.reshape(-1, x_train.shape[-1])
x_test_tmp = x_test.reshape(-1, x_test.shape[-1])

x_train_scaled = normalizer1.fit_transform(x_train_tmp)
x_test_scaled = normalizer2.fit_transform(x_test_tmp)

x_train = x_train_scaled.reshape(x_train.shape)
x_test = x_test_scaled.reshape(x_test.shape)

### Обучение модели

#### Модель для IBM

In [72]:
model = Sequential()

model.add(LSTM(250, activation='relu', return_sequences=True, input_shape=(n_steps_in, 5)))
model.add(Dropout(0.2))

model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(150, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=n_steps_out, activation='linear'))

model.compile(optimizer='adam', loss='mse')

history = model.fit(x_train, y_train, batch_size=15, epochs=60, shuffle=True, validation_split = 0.2)

Epoch 1/60
201/201 [==============================] - 34s 145ms/step - loss: 4160893184.0000 - val_loss: 56909.0078
Epoch 2/60
201/201 [==============================] - 29s 147ms/step - loss: 2612.6907 - val_loss: 384.1785
Epoch 3/60
201/201 [==============================] - 29s 146ms/step - loss: 760.0545 - val_loss: 494.2050
Epoch 4/60
201/201 [==============================] - 29s 143ms/step - loss: 620.5889 - val_loss: 309.6855
Epoch 5/60
201/201 [==============================] - 29s 145ms/step - loss: 631.0406 - val_loss: 657.6804
Epoch 6/60
201/201 [==============================] - 29s 145ms/step - loss: 614.8704 - val_loss: 1051.4255
Epoch 7/60
201/201 [==============================] - 29s 144ms/step - loss: 677.5818 - val_loss: 208.1014
Epoch 8/60
201/201 [==============================] - 29s 146ms/step - loss: 590.1005 - val_loss: 385.7183
Epoch 9/60
201/201 [==============================] - 29s 146ms/step - loss: 556.2628 - val_loss: 218.1200
Epoch 10/60
201/201 [=====

### Модель для Yandex

In [91]:
model = Sequential()

model.add(LSTM(250, activation='relu', return_sequences=True, input_shape=(n_steps_in, 5)))
model.add(Dropout(0.2))

model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(150, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=n_steps_out, activation='linear'))

model.compile(optimizer='rmsprop', loss='mse')

history = model.fit(x_train, y_train, batch_size=15, epochs=60, shuffle=True, validation_split = 0.2)

Epoch 1/60
77/77 [==============================] - 17s 158ms/step - loss: 3885.9834 - val_loss: 38.4388
Epoch 2/60
77/77 [==============================] - 11s 148ms/step - loss: 112.1995 - val_loss: 285.3660
Epoch 3/60
77/77 [==============================] - 11s 146ms/step - loss: 77.5695 - val_loss: 72.2024
Epoch 4/60
77/77 [==============================] - 11s 147ms/step - loss: 73.2149 - val_loss: 13.7916
Epoch 5/60
77/77 [==============================] - 11s 143ms/step - loss: 77.7692 - val_loss: 305.0338
Epoch 6/60
77/77 [==============================] - 11s 146ms/step - loss: 57.0193 - val_loss: 24.4391
Epoch 7/60
77/77 [==============================] - 11s 144ms/step - loss: 53.4754 - val_loss: 24.5159
Epoch 8/60
77/77 [==============================] - 11s 147ms/step - loss: 51.8838 - val_loss: 101.7112
Epoch 9/60
77/77 [==============================] - 11s 144ms/step - loss: 47.9301 - val_loss: 82.2166
Epoch 10/60
77/77 [==============================] - 11s 143ms/step

### Модель для MIME

In [119]:
model = Sequential()

model.add(LSTM(250, activation='relu', return_sequences=True, input_shape=(n_steps_in, 5)))
model.add(Dropout(0.2))

model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(150, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(150, activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=n_steps_out, activation='linear'))

model.compile(optimizer='rmsprop', loss='mse')

history = model.fit(x_train, y_train, batch_size=15, epochs=60, shuffle=True, validation_split = 0.2)

Epoch 1/60
28/28 [==============================] - 11s 202ms/step - loss: 2186346240.0000 - val_loss: 3690332.0000
Epoch 2/60
28/28 [==============================] - 5s 177ms/step - loss: 79393.2031 - val_loss: 1152045.3750
Epoch 3/60
28/28 [==============================] - 5s 183ms/step - loss: 50335.5742 - val_loss: 131955.7031
Epoch 4/60
28/28 [==============================] - 5s 170ms/step - loss: 5232.3076 - val_loss: 59069.1875
Epoch 5/60
28/28 [==============================] - 5s 172ms/step - loss: 1154.7003 - val_loss: 11458.0820
Epoch 6/60
28/28 [==============================] - 5s 171ms/step - loss: 649.7331 - val_loss: 4750.5674
Epoch 7/60
28/28 [==============================] - 5s 168ms/step - loss: 505.6823 - val_loss: 2004.5927
Epoch 8/60
28/28 [==============================] - 5s 168ms/step - loss: 398.7545 - val_loss: 14483.0830
Epoch 9/60
28/28 [==============================] - 5s 170ms/step - loss: 32918.6211 - val_loss: 8590.2900
Epoch 10/60
28/28 [=========

## Результаты

### Результаты для IBM

In [73]:
predict = model.predict(x_test)
predict

30/30 [==============================] - 3s 78ms/step


array([[187.13466 , 186.17651 , 189.35567 , ..., 192.22723 , 193.06573 ,
        189.91228 ],
       [185.01985 , 184.22049 , 187.42786 , ..., 190.13014 , 191.11105 ,
        187.9956  ],
       [182.09933 , 181.4907  , 184.78307 , ..., 187.23528 , 188.43167 ,
        185.33472 ],
       ...,
       [103.955185, 103.800415, 103.46153 , ..., 105.61159 , 105.86517 ,
        106.51098 ],
       [104.22279 , 104.04708 , 103.723755, ..., 105.88839 , 106.12563 ,
        106.7568  ],
       [104.43361 , 104.23998 , 103.89858 , ..., 106.0855  , 106.29806 ,
        106.93169 ]], dtype=float32)

In [74]:
y_test

array([[161.54, 162.67, 164.52, ..., 161.85, 162.34, 162.86],
       [162.67, 164.52, 164.05, ..., 162.34, 162.86, 162.38],
       [164.52, 164.05, 163.27, ..., 162.86, 162.38, 162.3 ],
       ...,
       [146.51, 145.97, 145.24, ..., 107.57, 111.39, 113.78],
       [145.97, 145.24, 145.37, ..., 111.39, 113.78, 113.03],
       [145.24, 145.37, 146.04, ..., 113.78, 113.03, 113.67]])

In [75]:
model.save('/content/LSTM_IBM_NICE.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Результаты для Yandex

In [92]:
predict = model.predict(x_test)
predict

12/12 [==============================] - 1s 77ms/step


array([[14.309579 , 14.42602  , 14.275345 , ..., 14.378533 , 14.0750065,
        13.759471 ],
       [14.29768  , 14.415672 , 14.2667675, ..., 14.364928 , 14.059179 ,
        13.747881 ],
       [14.303921 , 14.413824 , 14.276816 , ..., 14.270855 , 13.970999 ,
        13.664042 ],
       ...,
       [32.879723 , 33.064068 , 33.08372  , ..., 27.04904  , 26.923433 ,
        26.089102 ],
       [32.691784 , 32.88013  , 32.900482 , ..., 26.943312 , 26.798855 ,
        25.98207  ],
       [32.487823 , 32.680206 , 32.70089  , ..., 26.81879  , 26.656387 ,
        25.858612 ]], dtype=float32)

In [93]:
y_test

array([[22.41, 22.17, 22.43, ..., 29.53, 28.98, 28.98],
       [22.17, 22.43, 22.24, ..., 28.98, 28.98, 28.36],
       [22.43, 22.24, 22.06, ..., 28.98, 28.36, 29.16],
       ...,
       [31.63, 31.96, 31.49, ..., 27.08, 27.79, 27.56],
       [31.96, 31.49, 30.86, ..., 27.79, 27.56, 27.91],
       [31.49, 30.86, 31.79, ..., 27.56, 27.91, 27.35]])

In [94]:
model.save('/content/LSTM_YANDEX_NICE.h5')

### Результаты для MIME

In [120]:
predict = model.predict(x_test)
predict

5/5 [==============================] - 1s 73ms/step


array([[41.932415 , 39.290768 , 38.734787 , ..., 39.319942 , 50.896748 ,
        45.678814 ],
       [59.602142 , 54.211285 , 50.664845 , ..., 52.152924 , 69.8269   ,
        61.186275 ],
       [37.49378  , 42.84433  , 37.649113 , ..., 46.290573 , 51.90238  ,
        50.68556  ],
       ...,
       [14.8106365, 14.981062 , 14.887822 , ..., 20.080847 , 19.781507 ,
        19.996712 ],
       [17.274035 , 17.302937 , 16.89158  , ..., 21.64177  , 21.711983 ,
        21.541739 ],
       [18.993837 , 18.973372 , 18.26762  , ..., 22.896893 , 23.225367 ,
        22.853252 ]], dtype=float32)

In [121]:
y_test

array([[33.42, 33.23, 34.32, ..., 44.8 , 42.06, 40.91],
       [33.23, 34.32, 34.12, ..., 42.06, 40.91, 41.02],
       [34.32, 34.12, 34.01, ..., 40.91, 41.02, 39.47],
       ...,
       [38.57, 38.86, 39.08, ..., 30.87, 32.51, 32.7 ],
       [38.86, 39.08, 40.43, ..., 32.51, 32.7 , 33.32],
       [39.08, 40.43, 41.39, ..., 32.7 , 33.32, 33.63]])

In [122]:
model.save('/content/LSTM_MIME_NICE.h5')